In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, IntSlider

from src.data.dataset import split_dataset, tidy_plays_df
from src.data.data_query import StorageEngine
from src.data.plays_model import game_json_to_plays_list
from src.features.features import basic_features, advanced_features, normalize_plays_coords

In [ ]:
#tidied = tidy_plays_df([2015, 2016, 2017, 2018, 2019], augment=True)
#tidied.to_csv("./data/processed/plays_2015-2020.csv", index=False)

In [20]:
season_plays_df = pd.read_csv("./data/processed/plays_2015-2020.csv", index_col=False)
season_plays_df.head(15)

,event_idx,event_type_id,period_idx,period_type,game_time,period_time,shot_type,team_initiative_id,team_initiative_name,x_coord,...,previous_event_period,previous_event_period_time,previous_event_time,previous_event_type,previous_event_x_coord,previous_event_y_coord,gamePk,game_season,game_type,game_start_time
0,4,SHOT,1,REGULAR,2015-09-20T20:39:52Z,00:33,Snap Shot,NSH,Nashville Predators,-83.0,...,1.0,00:00,2015-09-20T20:38:24Z,FACEOFF,0.0,0.0,2015010002,20152016,PR,2015-09-20T20:30:00Z
1,16,SHOT,1,REGULAR,2015-09-20T20:42:37Z,01:36,Slap Shot,NSH,Nashville Predators,-34.0,...,1.0,01:18,2015-09-20T20:42:03Z,FACEOFF,-69.0,-22.0,2015010002,20152016,PR,2015-09-20T20:30:00Z
2,20,SHOT,1,REGULAR,2015-09-20T20:44:16Z,02:54,Slap Shot,NSH,Nashville Predators,-57.0,...,1.0,02:53,2015-09-20T20:44:15Z,GIVEAWAY,-97.0,-5.0,2015010002,20152016,PR,2015-09-20T20:30:00Z
3,23,SHOT,1,REGULAR,2015-09-20T20:45:28Z,03:41,Snap Shot,FLA,Florida Panthers,61.0,...,1.0,03:27,2015-09-20T20:44:49Z,HIT,88.0,-39.0,2015010002,20152016,PR,2015-09-20T20:30:00Z
4,24,SHOT,1,REGULAR,2015-09-20T20:45:23Z,04:00,Wrist Shot,FLA,Florida Panthers,64.0,...,1.0,03:41,2015-09-20T20:45:28Z,SHOT,61.0,4.0,2015010002,20152016,PR,2015-09-20T20:30:00Z
5,37,SHOT,1,REGULAR,2015-09-20T20:50:54Z,07:33,Wrist Shot,NSH,Nashville Predators,-76.0,...,1.0,07:20,2015-09-20T20:50:41Z,GIVEAWAY,-44.0,-26.0,2015010002,20152016,PR,2015-09-20T20:30:00Z
6,39,SHOT,1,REGULAR,2015-09-20T20:51:14Z,07:50,Wrist Shot,NSH,Nashville Predators,-41.0,...,1.0,07:40,2015-09-20T20:51:01Z,HIT,-57.0,38.0,2015010002,20152016,PR,2015-09-20T20:30:00Z
7,68,SHOT,1,REGULAR,2015-09-20T21:02:36Z,12:13,Wrist Shot,NSH,Nashville Predators,-55.0,...,1.0,11:55,2015-09-20T21:02:02Z,FACEOFF,-69.0,22.0,2015010002,20152016,PR,2015-09-20T20:30:00Z
8,70,GOAL,1,REGULAR,2015-09-20T21:04:07Z,13:43,Snap Shot,NSH,Nashville Predators,-69.0,...,1.0,12:29,2015-09-20T21:02:52Z,BLOCKED_SHOT,-74.0,3.0,2015010002,20152016,PR,2015-09-20T20:30:00Z
9,82,SHOT,1,REGULAR,2015-09-20T21:09:04Z,14:51,Slap Shot,NSH,Nashville Predators,-31.0,...,1.0,14:47,2015-09-20T21:08:59Z,MISSED_SHOT,-43.0,19.0,2015010002,20152016,PR,2015-09-20T20:30:00Z


In [3]:
advanced_features = advanced_features(season_plays_df)

In [17]:
@interact(row=IntSlider(options=advanced_features.shape[0]))
def view_features(row=0):
    example = advanced_features.iloc[row, :]
    
    plt.figure()
    plt.xlim(-100, 100)
    plt.ylim(-43, 43)
    plt.scatter(100-11, 0, c="red") # net
    plt.scatter(-(100-11), 0, c="red") # net
    plt.scatter(example.x_coord, example.y_coord, c="C0")
    plt.scatter(example.previous_x_coord, example.previous_y_coord, c="C1")
    plt.show()
    
    print("angle :", example.angle_from_net)
    print("dist :", example.dist_from_previous)
    print("seconds: ", example.seconds_from_previous)
    print("speed: ", example.speed)
    

interactive(children=(IntSlider(value=0, description='row'), Output()), _dom_classes=('widget-interact',))

In [23]:
plays_df = season_plays_df
plays_df = normalize_plays_coords(plays_df)
net_pos = np.array([100-11, 0])
shot_vector = net_pos - plays_df[["x_coord_norm", "y_coord_norm"]]
cos_angle = shot_vector @ net_pos / (np.linalg.norm(net_pos, ord=2) * np.linalg.norm(shot_vector, ord=2, axis=1))
angle = np.arccos(cos_angle)
np.degrees(angle)

0         65.224859
1          2.082565
2         42.184443
3          8.130102
4         15.642246
            ...    
437690    38.659808
437691    20.556045
437692     6.225829
437693     9.782407
437694     0.000000
Length: 437695, dtype: float64

In [41]:
game1 = plays_df.loc[plays_df.gamePk == 2015010002]
game1.team_initiative_id = game1.team_initiative_id.astype("category")